<a href="https://colab.research.google.com/github/MalyshevaVlada/kalambet/blob/main/praktos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

###УСТАНОВКА НЕОБХОДИМОГО

In [ ]:
%%bash
apt update &> /dev/null
apt install chromium-chromedriver &> /dev/null
pip install selenium &> /dev/null
pip install pymysql &> /dev/null

In [ ]:
import pymysql
import time
from pymysql import cursors
from logging import exception
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By

In [ ]:
options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('window-size=1400,800')
options.add_argument('user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/99.0.4844.51 Safari/537.36')

ПАРСЕР

In [ ]:
lst_product = []
lst_price = []
lst_link = []
lst_id = []
lst = []

for page in range(1,3):
  url = f'https://www.dns-shop.ru/catalog/17a8a01d16404e77/smartfony/?order=2&groupBy=avails&stock=now&p={page}'

  wd = webdriver.Chrome('chromedriver', options=options)
  wd.get(url)

  #изменение города поиска
  wd.refresh()
  wd.find_element(By.CSS_SELECTOR, 'i.location-icon').click()
  time.sleep(3)
  wd.find_element(By.LINK_TEXT, 'Южный').click()
  time.sleep(5)
  wd.find_element(By.LINK_TEXT, 'Ростовская область').click()
  time.sleep(8)
  wd.find_element(By.LINK_TEXT, 'Ростов-на-Дону').click()
  time.sleep(15)

  #хо-хо, вор ворует данные
  products_raw = wd.find_elements(By.CSS_SELECTOR, "a.catalog-product__name")
  for product in products_raw:
    lst_product.append(product.text)

  prices_raw = wd.find_elements(By.CSS_SELECTOR, "div.product-buy__price")
  for price in prices_raw:
    if "\n" in price.text:
      lst_price.append(price.text.split("\n")[0])
    else:
      lst_price.append(price.text)
  links_raw = wd.find_elements(By.CSS_SELECTOR, "a.catalog-product__name")
  for link in links_raw:
    lst_link.append(link.get_attribute("href"))

  ids_raw = wd.find_elements(By.CSS_SELECTOR, 'div.catalog-product.ui-button-widget')
  for id in ids_raw:
    lst_id.append(id.get_attribute('data-code'))

for i in range(31):
  Tp = (lst_id[i], lst_product[i], lst_price[i], lst_link[i])
  lst.append(Tp)
print(lst)

[('4867699', '6.7" Смартфон Apple iPhone 13 Pro Max 1000 ГБ серый [6x(1.8 ГГц, 3.2 ГГц), 1 SIM, OLED, 2778×1284, камера 12+12+12 Мп, NFC, 5G, GPS]', '184 999 ₽', 'https://www.dns-shop.ru/product/8c6be94615daed20/67-smartfon-apple-iphone-13-pro-max-1000-gb-seryj/'), ('4845714', '7.6" Смартфон Samsung Galaxy Z Fold3 512 ГБ черный [8x(2.84 ГГц), 12 Гб, 1 SIM, Dynamic AMOLED 2X, 1768х2208, камера 12+12+12 Мп, NFC, 5G, GPS, 4400 мА*ч]', '150 099 ₽', 'https://www.dns-shop.ru/product/5ecb692be84e2ff2/76-smartfon-samsung-galaxy-z-fold3-512-gb-cernyj/'), ('4867681', '6.1" Смартфон Apple iPhone 13 Pro 512 ГБ золотистый [6x(1.8 ГГц, 3.2 ГГц), 1 SIM, OLED, 2532х1170, камера 12+12+12 Мп, NFC, 5G, GPS]', '149 999 ₽', 'https://www.dns-shop.ru/product/8616d61715daed20/61-smartfon-apple-iphone-13-pro-512-gb-zolotistyj/'), ('4867680', '6.1" Смартфон Apple iPhone 13 Pro 512 ГБ серебристый [6x(1.8 ГГц, 3.2 ГГц), 1 SIM, OLED, 2532х1170, камера 12+12+12 Мп, NFC, 5G, GPS]', '149 999 ₽', 'https://www.dns-shop

MySQL

In [ ]:
try:
  connection = pymysql.connect(
        host = "gendalf.cf",
        port = 3308,
        user = "root",
        password = "1234567890",
        database = "malysheva",
        cursorclass = pymysql.cursors.DictCursor
  )
  print("Подключено")

  try:
    cursor = connection.cursor()


    #создание бд <------------------------
    try:
      create_database = "CREATE DATABASE malysheva"
      cursor.execute(create_database)
      print("БД создана")
    except Exception as ex:
      print("Ошибочка: ", ex)


    #переход к бд <------------------------
    try:
      cursor.execute("USE malysheva;")
      print("переход к базе данных выполнен")
    except Exception as ex:
      print("Ошибочка: ", ex)


    #создание таблицы <------------------------
    try:
      creat_table_query = "CREATE TABLE IdProductPriceLink (id int UNSIGNED AUTO_INCREMENT, idproduct VARCHAR(225), product VARCHAR(225), price VARCHAR(225), link VARCHAR(225), PRIMARY KEY(id))"
      cursor.execute(creat_table_query)
      print("Таблица создана")
    except Exception as ex:
      print("Ошибочка: ", ex)


    #добавление данных в таблицу <------------------------
    try:
      for i, v in enumerate(lst):
        insert_query = f"INSERT INTO IdProductPriceLink (idproduct, product, price, link) VALUES ({v[0]}, {v[1]}, {v[2]}, {v[3]})"
        cursor.execute(insert_query)
      connection.commit()
      print("данные добавлены в таблицу")
    except Exception as ex:
      print("Ошибочка: ", ex)


    #извлечение данных из таблицы <------------------------
    try:
      select_all_rows = "SELECT * FROM IdProductPriceLink"
      cursor.execute(select_all_rows)
      rows = cursor.fetchall()
      print("Данные: ")
      for row in rows:
        print(row)
    except Exception as ex:
      print("Ошибочка: ", ex)

  finally:
    connection.close()
    cursor.close()

except Exception as ex:
  print("Ошибочка: ", ex)

###Через csv в sql

In [ ]:
%%bash
apt update &> /dev/null
apt install chromium-chromedriver &> /dev/null
pip install selenium &> /dev/null

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import pandas as pd
from IPython.display import Image
from selenium.webdriver.common.keys import Keys
import time

In [ ]:
options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('window-size=1600,1200')
options.add_argument('user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.141 Safari/537.36')

In [ ]:
url_1='https://www.dns-shop.ru/catalog/17a8a01d16404e77/smartfony/?order=2&groupBy=avails&stock=now'
wd = webdriver.Chrome(options=options)

wd.get(url_1)
wd.refresh()
time.sleep(8)
wd.find_element(By.CSS_SELECTOR, 'i.location-icon').click()
time.sleep(8)
wd.find_element(By.LINK_TEXT, 'Южный').click()
time.sleep(5)
wd.find_element(By.LINK_TEXT, 'Ростовская область').click()
time.sleep(5)
wd.find_element(By.LINK_TEXT, 'Ростов-на-Дону').click()
time.sleep(10)


product_one = [i.text for i in wd.find_elements(By.CLASS_NAME, 'catalog-product__name')]
price_one = [price.text.split("\n")[0] for price in wd.find_elements(By.CLASS_NAME, 'product-buy__price')]
link_one = [i.get_attribute('href') for i in wd.find_elements(By.CSS_SELECTOR, 'a.catalog-product__name.ui-link.ui-link_black')]

id_one = [i.get_attribute('data-code') for i in wd.find_elements(By.CSS_SELECTOR, 'div.catalog-product.ui-button-widget')]

In [ ]:
url_2='https://www.dns-shop.ru/catalog/17a8a01d16404e77/smartfony/?order=2&groupBy=avails&stock=now&p=2'

wd = webdriver.Chrome(options=options)
wd.get(url_2)
wd.refresh()
time.sleep(7)
wd.find_element(By.CSS_SELECTOR, 'i.location-icon').click()
time.sleep(8)
wd.find_element(By.LINK_TEXT, 'Южный').click()
time.sleep(5)
wd.find_element(By.LINK_TEXT, 'Ростовская область').click()
time.sleep(5)
wd.find_element(By.LINK_TEXT, 'Ростов-на-Дону').click()
time.sleep(10)


product_two = [product.text for product in wd.find_elements(By.CLASS_NAME, 'catalog-product__name')][:12]
price_two = [price.text.split("\n")[0] for price in wd.find_elements(By.CLASS_NAME, 'product-buy__price')][:12]
link_two = [link.get_attribute('href') for link in wd.find_elements(By.CSS_SELECTOR, 'a.catalog-product__name.ui-link.ui-link_black')][:12]
id_two = [id.get_attribute('data-code') for id in wd.find_elements(By.CSS_SELECTOR, 'div.catalog-product.ui-button-widget')][:12]

In [ ]:
lst_id = id_one + id_two
lst_product = product_one + product_two
lst_price = price_one + price_two
lst_link = link_one + link_two


df=pd.DataFrame({
    'индекс': lst_id,
    'название': lst_product,
    'цена':lst_price,
    'ссылка': lst_link
})

df

In [ ]:
df.to_csv('malysheva.csv')

ЧТО-ТО ТИПО МУСОРА

In [ ]:
%%bash
apt update &> /dev/null
apt install chromium-chromedriver &> /dev/null
pip install selenium &> /dev/null
# pip install sqlalchemy &> /dev/null

In [ ]:
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

In [ ]:
options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('window-size=1600,1200')
options.add_argument('user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.141 Safari/537.36')

In [ ]:
# lst_product = []
# lst_price = []
# lst_link = []
# lst_id = []

url = f'https://www.dns-shop.ru/catalog/17a8a01d16404e77/smartfony/?order=2&groupBy=avails&stock=now&p=1'
wd = webdriver.Chrome('chromedriver', options=options)
wd.get(url)

  #изменение города поиска
wd.refresh()
time.sleep(5)
wd.find_element(By.CSS_SELECTOR, 'i.location-icon').click()
time.sleep(3)
wd.find_element(By.LINK_TEXT, 'Южный').click()
time.sleep(6)
wd.find_element(By.LINK_TEXT, 'Ростовская область').click()
time.sleep(8)
wd.find_element(By.LINK_TEXT, 'Ростов-на-Дону').click()
time.sleep(5)

  #хо-хо, вор ворует данные
products_raw = wd.find_elements(By.CSS_SELECTOR, "a.catalog-product__name")
product_one = [product.text for product in products_raw]
# for product in products_raw:
#     lst_product.append(product.text)

prices_raw = wd.find_elements(By.CSS_SELECTOR, "div.product-buy__price")
price_one=[price.text for price in prices_raw]
# for price in prices_raw:
#     if "\n" in price.text:
#       lst_price.append(price.text.split("\n")[0])
#     else:
#       lst_price.append(price.text)
links_raw = wd.find_elements(By.CSS_SELECTOR, "a.catalog-product__name")
link_one = [link.get_attribute("href") for link in links_raw]
# for link in links_raw:
#     lst_link.append(link.get_attribute("href"))

ids_raw = wd.find_elements(By.CSS_SELECTOR, 'div.catalog-product.ui-button-widget')
ids_one = [id.get_attribute('data-code') for id in ids_raw]
# for id in ids_raw:
#     lst_id.append(id.get_attribute('data-code'))

In [ ]:
url = f'https://www.dns-shop.ru/catalog/17a8a01d16404e77/smartfony/?order=2&groupBy=avails&stock=now&p=2'

  #хо-хо, вор ворует данные
  #хо-хо, вор ворует данные
products_raw = wd.find_elements(By.CSS_SELECTOR, "a.catalog-product__name")
product_two  = [product.text for product in products_raw]
# for product in products_raw:
#     lst_product.append(product.text)

prices_raw = wd.find_elements(By.CSS_SELECTOR, "div.product-buy__price")
price_two = [price.text for price in prices_raw]
# for price in prices_raw:
#     if "\n" in price.text:
#       lst_price.append(price.text.split("\n")[0])
#     else:
#       lst_price.append(price.text)
links_raw = wd.find_elements(By.CSS_SELECTOR, "a.catalog-product__name")
link_two = [link.get_attribute("href") for link in links_raw]
# for link in links_raw:
#     lst_link.append(link.get_attribute("href"))

ids_raw = wd.find_elements(By.CSS_SELECTOR, 'div.catalog-product.ui-button-widget')
ids_two = [id.get_attribute('data-code') for id in ids_raw]
# for id in ids_raw:
#     lst_id.append(id.get_attribute('data-code'))

In [ ]:
# sorted(lst_product, reverse=True)
# sorted(lst_price, reverse=True)
# sorted(lst_link, reverse=True)
# sorted(lst_id, reverse=True)

In [ ]:
df.to_csv('malysheva.csv')

In [ ]:
  lst_product = []
  lst_price = []
  lst_link = []
  lst_id = []

  url = f'https://www.dns-shop.ru/catalog/17a8a01d16404e77/smartfony/?order=2&groupBy=avails&stock=now&p=1'

  wd = webdriver.Chrome('chromedriver', options=options)
  wd.get(url)

  #изменение города поиска
  wd.refresh()
  wd.find_element(By.CSS_SELECTOR, 'i.location-icon').click()
  time.sleep(20)
  wd.find_element(By.LINK_TEXT, 'Южный').click()
  time.sleep(15)
  wd.find_element(By.LINK_TEXT, 'Ростовская область').click()
  time.sleep(8)
  wd.find_element(By.LINK_TEXT, 'Ростов-на-Дону').click()
  time.sleep(15)

  #хо-хо, вор ворует данные
  products_raw = wd.find_elements(By.CSS_SELECTOR, "a.catalog-product__name")
  for product in products_raw:
    lst_product.append(product.text)

  prices_raw = wd.find_elements(By.CSS_SELECTOR, "div.product-buy__price")
  for price in prices_raw:
    if "\n" in price.text:
      lst_price.append(price.text.split("\n")[0])
    else:
      lst_price.append(price.text)
  links_raw = wd.find_elements(By.CSS_SELECTOR, "a.catalog-product__name")
  for link in links_raw:
    lst_link.append(link.get_attribute("href"))

  ids_raw = wd.find_elements(By.CSS_SELECTOR, 'div.catalog-product.ui-button-widget')
  for id in ids_raw:
    lst_id.append(id.get_attribute('data-code'))

In [ ]:
from sqlalchemy import create_engine

engine = create_engine('sqlite://', echo=False)
df.to_sql('users', con=engine)
engine.execute("SELECT * FROM users").fetchall()

In [ ]:
Connection = sqlalchemy.create_engine("sqlite://malysheva.db")
df.to_sql('IdProductPriceLink', con=Connection)

In [ ]:
lst_product = []
lst_price = []
lst_link = []
lst_id = []

for page in range(1,3):
  url = f'https://www.dns-shop.ru/catalog/17a8a01d16404e77/smartfony/?order=2&groupBy=avails&stock=now&p={page}'

  wd = webdriver.Chrome('chromedriver', options=options)
  wd.get(url)

  #изменение города поиска
  wd.refresh()
  wd.find_element(By.CSS_SELECTOR, 'i.location-icon').click()
  time.sleep(20)
  wd.find_element(By.LINK_TEXT, 'Южный').click()
  time.sleep(15)
  wd.find_element(By.LINK_TEXT, 'Ростовская область').click()
  time.sleep(8)
  wd.find_element(By.LINK_TEXT, 'Ростов-на-Дону').click()
  time.sleep(15)

  #хо-хо, вор ворует данные
  products_raw = wd.find_elements(By.CSS_SELECTOR, "a.catalog-product__name")
  for product in products_raw:
    lst_product.append(product.text)

  prices_raw = wd.find_elements(By.CSS_SELECTOR, "div.product-buy__price")
  for price in prices_raw:
    if "\n" in price.text:
      lst_price.append(price.text.split("\n")[0])
    else:
      lst_price.append(price.text)
  links_raw = wd.find_elements(By.CSS_SELECTOR, "a.catalog-product__name")
  for link in links_raw:
    lst_link.append(link.get_attribute("href"))

  ids_raw = wd.find_elements(By.CSS_SELECTOR, 'div.catalog-product.ui-button-widget')
  for id in ids_raw:
    lst_id.append(id.get_attribute('data-code'))


In [ ]:
lst_id = id_one + id_two
lst_product = product_one + product_two
lst_price = price_one + price_two
lst_link = link_one + link_two

df=pd.DataFrame({
    'индекс': lst_id,
    'название': lst_product,
    'цена':lst_price,
    'ссылка': lst_link
})

df

,индекс,название,цена,ссылка
0,4845714,"7.6"" Смартфон Samsung Galaxy Z Fold3 512 ГБ че...",150 099 ₽\n164 999,https://www.dns-shop.ru/product/5ecb692be84e2f...
1,4867680,"6.1"" Смартфон Apple iPhone 13 Pro 512 ГБ сереб...",149 999 ₽,https://www.dns-shop.ru/product/8616d61615daed...
2,4867697,"6.7"" Смартфон Apple iPhone 13 Pro Max 512 ГБ з...",149 999 ₽,https://www.dns-shop.ru/product/8c6be94415daed...
3,4845724,"7.6"" Смартфон Samsung Galaxy Z Fold3 256 ГБ че...",140 999 ₽\n154 999,https://www.dns-shop.ru/product/2283dabde84f2f...
4,5010037,"6.6"" Смартфон Huawei P50 Pro 256 ГБ золотистый...",119 999 ₽,https://www.dns-shop.ru/product/76f0829b7f16ed...
5,5010042,"6.6"" Смартфон Huawei P50 Pro 256 ГБ черный [8x...",119 999 ₽,https://www.dns-shop.ru/product/046166517f17ed...
6,4800503,"6.5"" Смартфон Sony Xperia 1 III 256 ГБ черный ...",116 999 ₽,https://www.dns-shop.ru/product/a06a18cea0c72e...
7,5018254,"6.7"" Смартфон Apple iPhone 12 Pro Max ""Как нов...",109 999 ₽,https://www.dns-shop.ru/product/83db805c95d5ed...
8,4867610,"6.1"" Смартфон Apple iPhone 13 512 ГБ белый [6x...",109 199 ₽\n119 999,https://www.dns-shop.ru/product/52221a5315d2ed...
9,4867612,"6.1"" Смартфон Apple iPhone 13 512 ГБ красный [...",109 199 ₽\n119 999,https://www.dns-shop.ru/product/cb1c6d1715d3ed...
